####  Generate dummy raw CSV and inspect

In [2]:
import pandas as pd
import numpy as np
import os

# Ensure data folder exists
os.makedirs('data', exist_ok=True)

# Create 100 random expense records
np.random.seed(0)
dates   = pd.date_range('2023-01-01','2023-03-31',freq='D').to_pydatetime().tolist()
vendors = ['ABC Co.', 'ABC Company', 'abc co', 
           'XYZ Ltd', 'X.Y.Z. Ltd', 
           'Foo-Bar Inc', 'Foo Bar, Inc.']
categories    = ['Travel','Meals','Office Supplies', None]
payment_types = ['Credit Card','Cash','Bank Transfer']

rows = []
for _ in range(100):
    random_date = np.random.choice(dates)
    d   = random_date.strftime('%m/%d/%Y')
    v   = np.random.choice(vendors)
    cat = np.random.choice(categories, p=[0.3,0.3,0.3,0.1])
    amt = f"${np.random.randint(10,5000):,}.00"
    pay = np.random.choice(payment_types)
    rows.append([d, v, cat, amt, pay])

df_raw = pd.DataFrame(rows, columns=['Date','Vendor','Category','Amount','Payment'])
raw_csv = 'data/raw_expense_report.csv'
df_raw.to_csv(raw_csv, index=False)

# Show sample
print("Saved raw CSV to", raw_csv)
df_raw.head()


Saved raw CSV to data/raw_expense_report.csv


,Date,Vendor,Category,Amount,Payment
0,02/14/2023,Foo-Bar Inc,Office Supplies,"$4,869.00",Cash
1,03/25/2023,Foo-Bar Inc,Meals,"$3,478.00",Bank Transfer
2,03/07/2023,"Foo Bar, Inc.",Office Supplies,"$2,232.00",Credit Card
3,03/23/2023,Foo-Bar Inc,Office Supplies,"$2,173.00",Credit Card
4,03/11/2023,"Foo Bar, Inc.",Travel,"$2,506.00",Bank Transfer


#### Load & Inspect Raw Data

In [11]:
import pandas as pd

# Load the CSV created
df = pd.read_csv('data/raw_expense_report.csv')

# Preview the first 5 rows
print("🔍 Raw Data Preview:")
display(df.head())

# Check for missing values
print("\n⚠️ Missing Values by Column:")

print(df.isna().sum())

print("\n⚠️ data type:")

print(df.dtypes)

🔍 Raw Data Preview:


,Date,Vendor,Category,Amount,Payment
0,02/14/2023,Foo-Bar Inc,Office Supplies,"$4,869.00",Cash
1,03/25/2023,Foo-Bar Inc,Meals,"$3,478.00",Bank Transfer
2,03/07/2023,"Foo Bar, Inc.",Office Supplies,"$2,232.00",Credit Card
3,03/23/2023,Foo-Bar Inc,Office Supplies,"$2,173.00",Credit Card
4,03/11/2023,"Foo Bar, Inc.",Travel,"$2,506.00",Bank Transfer



⚠️ Missing Values by Column:
Date        0
Vendor      0
Category    7
Amount      0
Payment     0
dtype: int64

⚠️ data type:
Date        object
Vendor      object
Category    object
Amount      object
Payment     object
dtype: object


#### Clean Dates and Amounts

In [12]:
# Convert 'Date' from string to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# Strip '$' and ',' from 'Amount' and cast to float
df['Amount'] = (
    df['Amount']
      .str.replace(r'[\$,]', '', regex=True)
      .astype(float)
)

# Verify types and a quick summary
print("✅ Types after conversion:")
print(df.dtypes)
print("\n💰 Amount summary:")
print(df['Amount'].describe())

✅ Types after conversion:
Date        datetime64[ns]
Vendor              object
Category            object
Amount             float64
Payment             object
dtype: object

💰 Amount summary:
count     100.000000
mean     2408.290000
std      1377.152305
min        34.000000
25%      1203.500000
50%      2372.000000
75%      3429.500000
max      4994.000000
Name: Amount, dtype: float64


#### Normalize Vendor Names

In [13]:
import re

def normalize_vendor(v):
    v0 = str(v).strip().lower()
    if re.search(r'abc\s*(co|company)?', v0):
        return 'ABC Company'
    elif re.search(r'x[\W_]*y[\W_]*z\s*(ltd)?', v0):
        return 'XYZ Ltd'
    elif re.search(r'foo[\W_]*bar', v0):
        return 'Foo Bar Inc'
    return str(v).strip().title()

df['Vendor'] = df['Vendor'].apply(normalize_vendor)

print(df['Vendor'].unique())


['Foo Bar Inc' 'ABC Company' 'XYZ Ltd']


###### Here:
###### We maps all variations of “ABC Co.” → ABC Company, “XYZ Ltd” variants → XYZ Ltd, etc.

###### Title-cases any other vendor strings.

#### Fill Missing Categories & Flag Duplicates

In [14]:
# Fill blanks in Category
df['Category'] = df['Category'].fillna('Uncategorized')

# Flag exact duplicates on Date+Vendor+Amount
df['Duplicate'] = df.duplicated(subset=['Date','Vendor','Amount'], keep=False)

display(df.head())

,Date,Vendor,Category,Amount,Payment,Duplicate
0,2023-02-14,Foo Bar Inc,Office Supplies,4869.0,Cash,False
1,2023-03-25,Foo Bar Inc,Meals,3478.0,Bank Transfer,False
2,2023-03-07,Foo Bar Inc,Office Supplies,2232.0,Credit Card,False
3,2023-03-23,Foo Bar Inc,Office Supplies,2173.0,Credit Card,False
4,2023-03-11,Foo Bar Inc,Travel,2506.0,Bank Transfer,False


#### Saving the Final Cleaned Report

In [15]:
import os

output_csv = 'output/cleaned_expense_report.csv'
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
df.to_csv(output_csv, index=False)
print("✅ Cleaned data saved to:", output_csv)

✅ Cleaned data saved to: output/cleaned_expense_report.csv
